## Tests

In [1]:
#use this block to point to a local version of UWsubduction
import sys

try:
    sys.path.append('..')
except:
    pass

In [2]:
import os
import numpy as np
import underworld as uw
from underworld import function as fn
import glucifer
from easydict import EasyDict as edict
import networkx as nx
import operator
import UWsubduction as usub

In [3]:
#outputPath = os.path.join(os.path.abspath("."),"output/")
outputPath = os.path.join(os.path.abspath("."),"output/files")

if uw.rank()==0:
    if not os.path.exists(outputPath):
        os.makedirs(outputPath)
uw.barrier()

In [4]:
import UWsubduction.params as params 

#define some more concise names
ur = params.UnitRegistry
sca = params.sub_scaling
ndimlz = sca.nonDimensionalize
#build the dimensionless paramter / model dictionaries
ndp = params.build_nondim_dict(params.paramDict_dim  , sca)   
md = params.build_nondim_dict(params.modelDict_dim  , sca)

assert ndimlz(params.paramDict_dim.refLength) == 1.0

# changes to base params (for testing)
md.faultThickness *= 1.5 #15 km
md.res = 48
md.depth *= 0.85

/root/underworld2/unsupported/scaling/__init__.py:13: UserWarning: 

    The scaling module is not supported.

    It requires 'pint' as a dependency.

    You can install pint by running:

    'pip install pint' in a terminal

    Questions should be addressed to romain.beucher@unimelb.edu.au 
 
  Questions should be addressed to romain.beucher@unimelb.edu.au \n """


In [5]:
yres = int(md.res)
xres = int(md.res*6) 

halfWidth = 0.5*md.depth*md.aspectRatio 

minCoord_    = (-1.*halfWidth, 1. - md.depth) 
maxCoord_    = (halfWidth, 1.)

mesh = uw.mesh.FeMesh_Cartesian( elementType = (md.elementType),
                                 elementRes  = (xres, yres), 
                                 minCoord    = minCoord_, 
                                 maxCoord    = maxCoord_) 

velocityField = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=2)
pressureField   = uw.mesh.MeshVariable( mesh=mesh.subMesh, nodeDofCount=1 )
    

velocityField.data[:] = 0.
pressureField.data[:] = 0.

In [7]:
endTime = ndimlz(20.*ur.megayear)
dt = endTime                           #dummy value as we're onlu using the tectModel to set up the initial conds.
tm = usub.TectonicModel(mesh, 0, endTime, dt)

tm.add_plate(1)
tm.add_plate(2)
tm.add_plate(3)

In [14]:
tm.add_left_boundary(1)
tm.add_subzone(1, 2, md.subZoneLoc, subInitAge=md.slabAge, upperInitAge=md.opAgeAtTrench)
tm.add_subzone(3, 2., 0.4, subInitAge=md.slabAge, upperInitAge=md.opAgeAtTrench)
#tm.add_right_boundary(3, 0.)

In [15]:
for n in tm.nodes():
    bounds  = np.sort(tm.get_boundaries(n))
    print(bounds)

[-0.73275862 -0.03448276]
[-0.03448276  0.4       ]
plate does not have 2 boundaries. Cannot define extent.
[]


In [11]:
bounds

array([ 0.4       ,  0.73275862])